# Uniswap V2 LP Backtesting Demo

This notebook demonstrates vectorized backtesting of Uniswap V2 liquidity provider strategies using AMMBT.

We'll test thousands of strategy variants simultaneously to find optimal rebalancing parameters.

In [ ]:
import sys
sys.path.append('..')

import ammbt as amm
import numpy as np
import pandas as pd
from ammbt.plotting import plot_performance, plot_metrics_heatmap, plot_efficient_frontier, plot_pnl_distribution

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

## 1. Generate Synthetic Swap Data

Create realistic swap sequences with price movements and volume.

In [ ]:
# Generate 10,000 swaps with moderate volatility
swaps = amm.generate_swaps(
    n_swaps=10000,
    initial_price=1.0,
    volatility=0.03,       # 3% daily volatility
    drift=0.0005,          # Slight upward trend
    base_volume=50000,     # Average $50k per swap
    model='gbm',           # Geometric Brownian Motion
    seed=42,               # For reproducibility
)

print(f"Generated {len(swaps)} swaps")
print(f"Price range: {swaps['price'].min():.4f} - {swaps['price'].max():.4f}")
print(f"Total volume: ${swaps['volume'].sum():,.0f}")

swaps.head()

In [ ]:
# Quick plot of price path
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=swaps['price'], mode='lines', name='Price'))
fig.update_layout(
    title='Simulated Price Path',
    xaxis_title='Swap Index',
    yaxis_title='Price',
    template='plotly_dark',
    height=400,
)
fig.show()

## 2. Define Strategy Space

Test multiple combinations of:
- Initial capital
- Rebalancing thresholds (price deviation)
- Rebalancing frequency (time-based)

In [ ]:
# Create strategy grid
initial_capitals = [10_000, 25_000, 50_000, 100_000]
rebalance_thresholds = [0.0, 0.02, 0.05, 0.10, 0.15]  # 0%, 2%, 5%, 10%, 15%
rebalance_frequencies = [0, 100, 500, 1000]  # Never, every 100/500/1000 swaps

# Generate all combinations
strategies = []
for capital in initial_capitals:
    for threshold in rebalance_thresholds:
        for frequency in rebalance_frequencies:
            strategies.append({
                'initial_capital': capital,
                'rebalance_threshold': threshold,
                'rebalance_frequency': frequency,
            })

strategy_df = pd.DataFrame(strategies)

print(f"Testing {len(strategy_df)} strategy variants")
print(f"\nStrategy space:")
print(f"  Capital: {initial_capitals}")
print(f"  Thresholds: {rebalance_thresholds}")
print(f"  Frequencies: {rebalance_frequencies}")

strategy_df.head(10)

## 3. Run Backtest

Simulate all strategies simultaneously using vectorized operations.

In [ ]:
%%time

# Initialize backtester
backtester = amm.LPBacktester(
    amm_type='v2',
    initial_reserve0=1_000_000,
    initial_reserve1=1_000_000,
    fee_rate=0.003,  # 0.3%
)

# Run simulation
results = backtester.run(swaps, strategy_df)

print(results)

## 4. Analyze Results

Examine performance metrics across all strategies.

In [ ]:
# Top 10 strategies by net PnL
top_strategies = results.summary().head(10)

print("Top 10 Strategies by Net PnL:")
print("=" * 80)
top_strategies[[
    'net_pnl', 
    'total_return_pct', 
    'il_pct', 
    'total_fees', 
    'sharpe',
    'num_rebalances',
]]

In [ ]:
# Summary statistics
print("Performance Statistics:")
print("=" * 80)
print(f"Mean Net PnL: ${results.metrics['net_pnl'].mean():,.2f}")
print(f"Best Net PnL: ${results.metrics['net_pnl'].max():,.2f}")
print(f"Worst Net PnL: ${results.metrics['net_pnl'].min():,.2f}")
print(f"\nMean IL: {results.metrics['il_pct'].mean():.2f}%")
print(f"Mean Fees: ${results.metrics['total_fees'].mean():,.2f}")
print(f"Mean Sharpe: {results.metrics['sharpe'].mean():.2f}")
print(f"\nStrategies with positive PnL: {(results.metrics['net_pnl'] > 0).sum()} / {len(results.metrics)}")

## 5. Visualizations

In [ ]:
# Plot best strategy performance
best_idx = results.metrics['net_pnl'].idxmax()

print(f"Best Strategy (Index {best_idx}):")
print(strategy_df.loc[best_idx])
print(f"\nNet PnL: ${results.metrics.loc[best_idx, 'net_pnl']:,.2f}")

fig = plot_performance(results, strategy_idx=best_idx)
fig.show()

In [ ]:
# Heatmap: Net PnL across rebalance parameters (for $50k capital)
capital_50k_mask = strategy_df['initial_capital'] == 50_000
subset_results_df = pd.concat([strategy_df, results.metrics], axis=1)
subset_results_df = subset_results_df[capital_50k_mask]

heatmap_data = subset_results_df.pivot_table(
    values='net_pnl',
    index='rebalance_threshold',
    columns='rebalance_frequency',
    aggfunc='mean',
)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlGn',
    colorbar=dict(title='Net PnL'),
))

fig.update_layout(
    title='Net PnL Heatmap ($50k Capital)',
    xaxis_title='Rebalance Frequency (swaps)',
    yaxis_title='Rebalance Threshold',
    template='plotly_dark',
    height=500,
)
fig.show()

In [ ]:
# Efficient frontier
fig = plot_efficient_frontier(
    results,
    risk_metric='max_drawdown_pct',
    return_metric='total_return_pct',
)
fig.show()

In [ ]:
# PnL distribution
fig = plot_pnl_distribution(results)
fig.show()

## 6. Strategy Comparison

Compare different rebalancing approaches.

In [ ]:
# Group by rebalancing strategy type
combined = pd.concat([strategy_df, results.metrics], axis=1)

# Never rebalance
never_rebalance = combined[
    (combined['rebalance_threshold'] == 0) & 
    (combined['rebalance_frequency'] == 0)
]

# Price-based rebalancing (5% threshold)
price_rebalance = combined[
    (combined['rebalance_threshold'] == 0.05) & 
    (combined['rebalance_frequency'] == 500)
]

# Time-based rebalancing (every 500 swaps)
time_rebalance = combined[
    (combined['rebalance_threshold'] == 0) & 
    (combined['rebalance_frequency'] == 500)
]

comparison = pd.DataFrame({
    'Strategy': ['Never Rebalance', 'Price-Based (5%)', 'Time-Based (500)'],
    'Mean PnL': [
        never_rebalance['net_pnl'].mean(),
        price_rebalance['net_pnl'].mean(),
        time_rebalance['net_pnl'].mean(),
    ],
    'Mean Sharpe': [
        never_rebalance['sharpe'].mean(),
        price_rebalance['sharpe'].mean(),
        time_rebalance['sharpe'].mean(),
    ],
    'Avg Rebalances': [
        never_rebalance['num_rebalances'].mean(),
        price_rebalance['num_rebalances'].mean(),
        time_rebalance['num_rebalances'].mean(),
    ],
    'Avg Gas Costs': [
        never_rebalance['gas_costs'].mean(),
        price_rebalance['gas_costs'].mean(),
        time_rebalance['gas_costs'].mean(),
    ],
})

print("\nRebalancing Strategy Comparison:")
print("=" * 80)
comparison

## 7. Key Insights

Analyze what we learned from testing thousands of strategies.

In [ ]:
# Correlation analysis
combined_analysis = pd.concat([strategy_df, results.metrics], axis=1)

correlations = combined_analysis[[
    'initial_capital',
    'rebalance_threshold',
    'rebalance_frequency',
    'net_pnl',
    'sharpe',
    'il_pct',
]].corr()

print("Correlation Matrix:")
print("=" * 80)
correlations

## Conclusion

This demo shows how AMMBT enables:
- **Vectorized backtesting**: Test thousands of strategies in seconds
- **Strategy optimization**: Find optimal rebalancing parameters
- **Risk analysis**: Understand IL, fees, and total return tradeoffs

Next steps:
- Test with real swap data
- Add Uniswap v3 (concentrated liquidity)
- Add Meteora DLMM (bin-based liquidity)